In [2]:
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from ISLP.models import poly, summarize

In [3]:
import inspect

In [4]:
print(inspect.getsource(load_data))

def load_data(dataset):
    if dataset == 'NCI60':
        with as_file(files('ISLP').joinpath('data', 'NCI60data.npy')) as features:
            X = np.load(features)
        with as_file(files('ISLP').joinpath('data', 'NCI60labs.csv')) as labels:
            Y = pd.read_csv(labels)
        return {'data':X, 'labels':Y}
    elif dataset == 'Khan':
        with as_file(files('ISLP').joinpath('data', 'Khan_xtest.csv')) as xtest:
            xtest = pd.read_csv(xtest)
        xtest = xtest.rename(columns=dict([('V%d' % d, 'G%04d' % d) for d in range(1, len(xtest.columns)+0)]))
        with as_file(files('ISLP').joinpath('data', 'Khan_ytest.csv')) as ytest:
            ytest = pd.read_csv(ytest)
        ytest = ytest.rename(columns={'x':'Y'})
        ytest = ytest['Y']
        
        with as_file(files('ISLP').joinpath('data', 'Khan_xtrain.csv')) as xtrain:
            xtrain = pd.read_csv(xtrain)
            xtrain = xtrain.rename(columns=dict([('V%d' % d, 'G%04d' % d) for d in range(1

In [5]:
print(inspect.getsource(MS))

class ModelSpec(TransformerMixin, BaseEstimator):

    '''

    Parameters
    ----------

    terms : sequence (optional)
        Sequence of sets whose
        elements are columns of *X* when fit.
        For :py:class:`pd.DataFrame` these can be column
        names.

    intercept : bool (optional)
        Include a column for intercept?

    categorical_features : array-like of {bool, int} of shape (n_features) 
            or shape (n_categorical_features,), default=None.
        Indicates the categorical features. Will be ignored if *X* is a :py:class:`pd.DataFrame`
        or :py:class:`pd.Series`.

        - None : no feature will be considered categorical for :py:class:`np.ndarray`.
        - boolean array-like : boolean mask indicating categorical features.
        - integer array-like : integer indices indicating categorical
          features.

    default_encoders : dict
        Dictionary whose keys are elements of *terms* and values
        are transforms to be applied

In [6]:
print(inspect.getsource(poly))

def poly(col,
         degree=1,
         intercept=False,
         raw=False,
         name=None):

    """
    Create a polynomial Feature
    for a given column.
    
    Additional `args` and `kwargs`
    are passed to `Poly`.

    Parameters
    ----------

    col : column identifier or Column
        Column to transform.

    degree : int, default=1
        Degree of polynomial.

    intercept : bool, default=False
        Include a column for intercept?

    raw : bool, default=False
        If False, perform a QR decomposition on the resulting
        matrix of powers of centered and / or scaled features.

    name : str (optional)
        Defaults to one derived from col.

    Returns
    -------

    var : Feature
    """
    shortname, klass = 'poly', Poly
    encoder = klass(degree=degree,
                    raw=raw,
                    intercept=intercept) 
    if name is None:
        if isinstance(col, Column):
            name = col.name
        else:
            name

In [7]:
print(inspect.getsource(summarize))

def summarize(results,
              conf_int=False):
    """
    Take a fit statsmodels and summarize it
    by returning the usual coefficient estimates,
    their standard errors, the usual test
    statistics and P-values as well as 
    (optionally) 95% confidence intervals.

    Based on:

    https://stackoverflow.com/questions/51734180/converting-statsmodels-summary-object-to-pandas-dataframe

    Parameters
    ----------

    results : a results object

    conf_int : bool (optional)
        Include 95% confidence intervals?

    """
    tab = results.summary().tables[1]
    results_table = pd.read_html(tab.as_html(),
                                 index_col=0,
                                 header=0)[0]
    if not conf_int:
        columns = ['coef',
                   'std err',
                   't',
                   'P>|t|']
        return results_table[results_table.columns[:-2]]
    return results_table

